In [84]:
import pandas as pd

# READ DATASETS

In [86]:
folder = "./datasets/"


### STATIONS DATASET


In [104]:
df_stations = pd.read_csv(folder + "DB_estaciones.txt", sep=";", header=0)

In [105]:

df_mesures = pd.read_csv(folder + "datos_hume.csv", header=0).drop(['Unnamed: 0'], axis=1)

In [106]:
df_mesures

,id,codi_estacio,codi_variable,data_lectura,valor_lectura
0,CA330101090000,CA,33,2009-01-01T00:00:00.000,85.0
1,CC330101090000,CC,33,2009-01-01T00:00:00.000,94.0
2,CD330101090000,CD,33,2009-01-01T00:00:00.000,93.0
3,CE330101090000,CE,33,2009-01-01T00:00:00.000,88.0
4,CG330101090000,CG,33,2009-01-01T00:00:00.000,50.0
...,...,...,...,...,...
22356006,Z8332805232200,Z8,33,2023-05-28T22:00:00.000,100.0
22356007,Z9332805232200,Z9,33,2023-05-28T22:00:00.000,92.0
22356008,ZB332805232200,ZB,33,2023-05-28T22:00:00.000,85.0
22356009,ZC332805232200,ZC,33,2023-05-28T22:00:00.000,87.0


# PROCESS DATA

In [123]:
codiE_por_comarca = {}
for comarca, group in df_stations.groupby('comarca'):
    codiE_por_comarca[comarca] = list(group['codiE'].unique())

In [124]:
comarcas_bcn = ['Vallès Oriental', 'Alt Penedès', 'Baix Llobregat', 'Barcelonès', 'Garraf', 'Maresme', 'Vallès Occidental']
codiE_comarca = [codiE_por_comarca[i] for i in comarcas_bcn]

In [125]:
data_comarca = []
for i in comarcas_bcn:
    df_filtrado = df_mesures[df_mesures['codi_estacio'].isin(codiE_por_comarca[i])]
    data_comarca.append(df_filtrado.reset_index(drop=True))

In [126]:
for comarca in data_comarca:
    comarca.data_lectura = comarca.data_lectura.str.split('T').str[0]

In [127]:
media_por_dia_comarca = []
for comarca in data_comarca:
    comarca['data_lectura'] = pd.to_datetime(comarca['data_lectura'])
    media_por_dia_comarca.append(comarca.groupby([comarca['data_lectura'].dt.date, 'codi_variable'])['valor_lectura'].mean().reset_index())

In [128]:
nuevo_orden = ["comarca", "codi_variable", "data_lectura", "valor_lectura"]
for i in range(len(media_por_dia_comarca)):
    media_por_dia_comarca[i]["comarca"] = comarcas_bcn[i]
    media_por_dia_comarca[i] = media_por_dia_comarca[i][nuevo_orden].reset_index(drop=True)

In [129]:
df_concatenado = pd.concat(media_por_dia_comarca).reset_index(drop=True)
df_concatenado.to_csv(folder + "mean_hume_x_mes.csv")

In [130]:
df_concatenado

,comarca,codi_variable,data_lectura,valor_lectura
0,Vallès Oriental,33,2009-01-01,86.152778
1,Vallès Oriental,33,2009-01-02,91.291667
2,Vallès Oriental,33,2009-01-03,92.083333
3,Vallès Oriental,33,2009-01-04,81.638889
4,Vallès Oriental,33,2009-01-05,76.811765
...,...,...,...,...
37095,Vallès Occidental,33,2023-07-05,84.946429
37096,Vallès Occidental,33,2023-07-06,76.791667
37097,Vallès Occidental,33,2023-07-07,68.672619
37098,Vallès Occidental,33,2023-07-08,61.208333
